In [6]:
# stream data

import pandas as pd
import numpy as np

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

arrestData = pd.read_csv("PPAD.csv")

nPopulations = pd.read_csv("totalPopulation.csv", index_col = 'Id')

eduA = pd.read_csv("eduAttainment.csv")


#eduA.head(10)

In [2]:
# create dict of neighborhoods w default vals
nc = {}

for i in range(1, len(nPopulations)):
    nValue = nPopulations.loc[i, "Neighborhood"]
    if(nValue not in nc):
        nc[nValue] = 0
#print(nc)  

In [3]:
# create dict of neighborhoods : populations

numOfNeighborhoods = 0

for i in range(len(nPopulations)):
    numOfNeighborhoods += 1

pop = {}

for i in range(1, len(nPopulations)):
    val = nPopulations.loc[i, "Neighborhood"]
    if(val in nPopulations.values):
        pop[val] = nPopulations.get_value(nPopulations.loc[nPopulations['Neighborhood'] == val].index[0], 'Estimate; Total')

#pop

/ihome/crc/install/python/miniconda3-3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


In [8]:
# create dict of neighborhoods : crimes

for i in range(len(arrestData)):
    nValue = arrestData.loc[i, "INCIDENTNEIGHBORHOOD"]
    if(nValue in nc):
        nc[nValue] += 1
#nc   